# P2 - Continuous Control

---

In this notebook, I will describe my solution to the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Learning Algorithm.

I have chosen the multi-agent version of the problem, as I wanted to apply A3C (Asynchronous Advantage Actor Critic) to it. Using this multi-agent approach, we also remove the need to use a replay memory, as the model is getting its uncorrelated data from the various agents that are trying to solve the problem at once. This sounded like a better approach to being able to apply any solution like this to my field of work (AAA videogame industry).


### 2. Implementation



In [1]:
from unityagents import UnityEnvironment
import numpy as np
from agent import AgentOverlord
import matplotlib.pyplot as plt

# Initialize the environment - please note we're using v2 of the problem (multiple agents)
env = UnityEnvironment( file_name='Reacher_Windows_x86_64/Reacher.exe' )

# Get brain - this will be used to control the unity environment
brain_name = env.brain_names[ 0 ]
brain = env.brains[ brain_name ]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


#### 2.1. Agent initialization

From the description of the problem, we know that:

> "In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible. 
The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`."

With this information, let's initialize our agents and get prepared to solve the problem (in this implementation, we are actually adding an "agent overlord" that will keep track of all of our agents).


In [2]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]

# number of agents
agent_num = states.shape[ 0 ]
print('Number of agents:', agent_num)

# hyperparameters
actor_layer_size=128
critic_layer_size = 128
gamma = 0.99
learning_rate=5e-4

# initialize overlord
agent_overlord = AgentOverlord( state_size, action_size, agent_num, actor_layer_size, critic_layer_size, gamma, learning_rate )


Size of each action: 4
Number of agents: 20


### 3. Solve the problem

We are now ready to start solving the problem. To do so, we will reset our environment and step forever.

In [ ]:
episode = 1
meanScores = []

while True:
    # let's restart the environment and get the initial states of the agents
    env_info = env.reset( train_mode=True )[ brain_name ]    
    states = env_info.vector_observations
    episodeScores = np.zeros(agent_num)

    while True:

        # choose the actions we should take
        actions = agent_overlord.choose_actions( states )

        # apply them
        env_info = env.step( actions )[ brain_name ]

        next_states = env_info.vector_observations

        rewards = env_info.rewards
        dones = env_info.local_done

        # update the overlord
        agent_overlord.step( states, actions, rewards, dones )

        # update states for next iteration
        states = next_states

        episodeScores += env_info.rewards

        if np.any( dones ):
            # episode is done
            break

    episodeMeanScore = np.mean(episodeScores)
    meanScores.append( episodeMeanScore )
    agent_overlord.clear()
    
    print('Episode {:d} - Total score (averaged over agents): {:.2f}'.format(episode, episodeMeanScore))
    meanTotalScore = np.mean( meanScores )
    if ( meanTotalScore >= 30 ):
        print('Episode count was {}', format( episode ) )
        print('Mean score was {}', meanTotalScore )
        break
    
    episode = episode + 1
    if episode > 300:
        break
    
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(meanScores)), meanScores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show

Episode 1 - Total score (averaged over agents): 0.12
Episode 2 - Total score (averaged over agents): 0.08
Episode 3 - Total score (averaged over agents): 0.05
Episode 4 - Total score (averaged over agents): 0.18
Episode 5 - Total score (averaged over agents): 0.19
Episode 6 - Total score (averaged over agents): 0.17
Episode 7 - Total score (averaged over agents): 0.09
Episode 8 - Total score (averaged over agents): 0.07
Episode 9 - Total score (averaged over agents): 0.18
Episode 10 - Total score (averaged over agents): 0.09
Episode 11 - Total score (averaged over agents): 0.15
Episode 12 - Total score (averaged over agents): 0.14
Episode 13 - Total score (averaged over agents): 0.13
Episode 14 - Total score (averaged over agents): 0.07
Episode 15 - Total score (averaged over agents): 0.16
Episode 16 - Total score (averaged over agents): 0.12
Episode 17 - Total score (averaged over agents): 0.18
Episode 18 - Total score (averaged over agents): 0.15
Episode 19 - Total score (averaged ov

When finished, you can close the environment.

In [ ]:
env.close()